In [1]:
from pyspark import SparkContext
sc = SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/26 12:32:33 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
pythonList = [1,2,3,4,5,6,7,8,9,10]
pythonList2 = [('s1',1),('s2',2),('s1',3),('s4',4),('s2',5),('s4',6)]

In [3]:
rdd = sc.parallelize(pythonList, 2)
rdd2 = sc.parallelize(pythonList2, 2)

In [4]:
rdd.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [5]:
rdd2.collect()

[('s1', 1), ('s2', 2), ('s1', 3), ('s4', 4), ('s2', 5), ('s4', 6)]

In [6]:
# Aggregate sum
rdd.aggregate((0,0), seqOp= (lambda local_part, local_length: (local_part[0] + local_length, local_part[1]+1)) \
              , combOp= (lambda total_local_part, total_local_length: (total_local_part[0] + total_local_length[0], \
                                                                      total_local_part[1] + total_local_length[1])))

(55, 10)

In [31]:
rdd.aggregate(0, seqOp= (lambda data, data2: (data + data2)) \
              ,combOp= (lambda data, data2: (data + data2)))

55

https://stackoverflow.com/questions/28240706/explain-the-aggregate-functionality-in-spark-with-python-and-scala

In [71]:
#average
rdd.aggregate((0, 0), seqOp= (lambda local_part, local_length: ((local_part[0]+local_length), (local_part[1]+1))) \
              , combOp= (lambda totol_local_part, total_local_length: ((totol_local_part[0] + total_local_length[0]), \
                                                                       totol_local_part[1]+total_local_length[1]))) \
                

(55, 10)

In [32]:
rdd.aggregate(0, seqOp=(lambda data, data2: (data + data2)/2) \
             ,combOp=(lambda data, data2: (data + data2)/2))

5.4453125

In [14]:
#min
rdd.aggregate((1,0), seqOp=(lambda local_part, local_length: (min(local_part[0], local_length), local_part[1]+1)) \
             ,combOp=(lambda total_local_part,total_local_length: (min(total_local_part[0], total_local_length[0]), \
                                                                  total_local_part[1]+total_local_length[1])))

(1, 10)

In [33]:
rdd.aggregate(50000000000000000000000000000000000000000000000000000000000000000000000,
             seqOp= (lambda data, data2: min(data, data2)) \
             ,combOp=(lambda data, data2: min(data, data2)))

1

In [15]:
#max
rdd.aggregate((0,0), seqOp=(lambda local_part, local_length: (max(local_part[0], local_length), local_part[1]+1)) \
             ,combOp=(lambda total_local_part,total_local_length: (max(total_local_part[0], total_local_length[0]), \
                                                                  total_local_part[1]+total_local_length[1])))

(10, 10)

In [34]:
rdd.aggregate(0, seqOp=(lambda data, data2: max(data, data2)) \
             , combOp=(lambda data, data2: max(data, data2)))

10

In [19]:
## Aggregateby keys sum

In [30]:
rdd2.aggregateByKey(0, seqFunc=(lambda x,y: (x+y)), combFunc=(lambda z,c:(z+c))).collect()

[('s1', 4), ('s2', 7), ('s4', 10)]

In [67]:
rdd2.aggregateByKey((0,0),(lambda local_part, local_length: [local_part[0] + local_length, local_part[1]+1]) \
              ,(lambda total_local_part, total_local_length: [total_local_part[0] + total_local_length[0], \
                                                                      total_local_part[1] + total_local_length[1]])) \
.collect()

[('s1', [4, 2]), ('s2', [7, 2]), ('s4', [10, 2])]

In [78]:
#average
rdd2.aggregateByKey((0,0), seqFunc=(lambda acc, part: [acc[0]+part, acc[1]+1]) \
                   ,combFunc=(lambda local_part, another_local_part: [local_part[0]+another_local_part[0] \
                                                                     ,local_part[1]+another_local_part[1]])) \
.map(lambda data: (data[0], data[1][0]/data[1][1])).collect()

[('s1', 2.0), ('s2', 3.5), ('s4', 5.0)]

In [79]:
#max
rdd2.aggregateByKey(0, seqFunc=(lambda acc, data:(max(acc,data))), 
                   combFunc=(lambda local_part, another_local_part: max(local_part, another_local_part))).collect()

[('s1', 3), ('s2', 5), ('s4', 6)]

In [81]:
#min
rdd2.aggregateByKey(10, seqFunc=(lambda acc, data:min(acc, data)),
                   combFunc=(lambda acc, data: min(acc, data))).collect()

[('s1', 1), ('s2', 2), ('s4', 4)]

In [4]:
filDataSingle = [['filamentA','100W',605],
                  ['filamentB','100W',683],
                  ['filamentB','100W',691],
                  ['filamentB','200W',561],
                  ['filamentA','200W',530],
                  ['filamentA','100W',619],
                  ['filamentB','100W',686],
                  ['filamentB','200W',600],
                  ['filamentB','100W',696],
                  ['filamentA','200W',579],
                  ['filamentA','200W',520],
                  ['filamentA','100W',622],
                  ['filamentA','100W',668],
                  ['filamentB','200W',569],
                  ['filamentB','200W',555],
                  ['filamentA','200W',541]]
filDataSingleRDD = sc.parallelize(filDataSingle,2)
filDataSingleRDD.take(3)

[['filamentA', '100W', 605],
 ['filamentB', '100W', 683],
 ['filamentB', '100W', 691]]

In [5]:
complexRDD = filDataSingleRDD.map(lambda data: [(data[0], data[1]), data[2]])

In [28]:
complexRDD.take(2)

[[('filamentA', '100W'), 605], [('filamentB', '100W'), 683]]

In [86]:
#sum on complex rdd
complexRDD.aggregateByKey(0, seqFunc=(lambda acc, data: acc+data),
                         combFunc=(lambda local_part, another_local_part: local_part+another_local_part)).collect()

[(('filamentB', '100W'), 2756),
 (('filamentA', '200W'), 2170),
 (('filamentA', '100W'), 2514),
 (('filamentB', '200W'), 2285)]

In [89]:
#Average on complex rdd
complexRDD.aggregateByKey((0,0), seqFunc=(lambda acc, data: [acc[0]+data, acc[1]+1]),
                         combFunc=(lambda local_part, another_local_part: [local_part[0]+another_local_part[0] \
                                                                           , local_part[1]+another_local_part[1]])) \
.map(lambda data: [data[0], data[1][0]/data[1][1]]).collect()

[[('filamentB', '100W'), 689.0],
 [('filamentA', '200W'), 542.5],
 [('filamentA', '100W'), 628.5],
 [('filamentB', '200W'), 571.25]]

In [92]:
#min on complex rdd
complexRDD.aggregateByKey(50000, seqFunc=(lambda acc, data: min(acc, data)),
                         combFunc=(lambda local_part, another_local_part: min(local_part, another_local_part))) \
.collect()

[(('filamentB', '100W'), 683),
 (('filamentA', '200W'), 520),
 (('filamentA', '100W'), 605),
 (('filamentB', '200W'), 555)]

In [93]:
#max on complex rdd
complexRDD.aggregateByKey(0, seqFunc=lambda acc, data: max(acc, data),
                         combFunc=lambda local_part, another_local_part: max(local_part, another_local_part)) \
.collect()

[(('filamentB', '100W'), 696),
 (('filamentA', '200W'), 579),
 (('filamentA', '100W'), 668),
 (('filamentB', '200W'), 600)]

### Cache

PySpark cache() method is used to cache the intermediate results of the transformation into memory so that any future transformations on the results of cached transformation improve the performance. Caching is a lazy evaluation meaning it will not cache the results until you call the action operation and the result of the transformation is one of the optimization tricks to improve the performance of the long-running PySpark applications/jobs.

https://stackoverflow.com/questions/38581809/what-is-the-purpose-of-cache-an-rdd-in-apache-spark

In [94]:
rdd2.collect()

[('s1', 1), ('s2', 2), ('s1', 3), ('s4', 4), ('s2', 5), ('s4', 6)]

In [100]:
rdd2.cache().collect()

[('s1', 1), ('s2', 2), ('s1', 3), ('s4', 4), ('s2', 5), ('s4', 6)]

In [101]:
rdd2.collect()

[('s1', 1), ('s2', 2), ('s1', 3), ('s4', 4), ('s2', 5), ('s4', 6)]

#### pyspark.RDD.cartesian

cartesian() in RDD will return the new RDD that includes all the elements from both the RDDs. It returns a cartesian product such that each element in the first RDD is combined with all elements from the second RDD in the form of a pair.

In [105]:
rdd5 = sc.parallelize([1,2,3])

In [106]:
rdd5.cartesian(rdd5).collect()

[(1, 1), (1, 2), (1, 3), (2, 1), (3, 1), (2, 2), (2, 3), (3, 2), (3, 3)]

In [108]:
rdd.cartesian(rdd5).take(5)

[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]

#### RDD CheckPointing
Mark this RDD for checkpointing. It will be saved to a file inside the checkpoint directory set with SparkContext.setCheckpointDir() and all references to its parent RDDs will be removed. This function must be called before any job has been executed on this RDD. It is strongly recommended that this RDD is persisted in memory, otherwise saving it on a file will require recomputation.

In [110]:
sc.setCheckpointDir('/user/spark_checkpointing/')

22/12/24 15:59:53 WARN SparkContext: Spark is not running in local mode, therefore the checkpoint directory must not be on the local filesystem. Directory '/user/spark_checkpointing/' appears to be on the local filesystem.


In [111]:
rdd5.checkpoint()

In [113]:
rdd5.is_checkpointed

True

https://livebook.manning.com/book/spark-in-action-with-examples-in-java/16-cache-and-checkpoint-enhancing-spark-s-performances/v-14/50

#### pyspark.RDD.coalesce

In [120]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10],4)
rdd.glom().collect()

[[1, 2], [3, 4], [5, 6], [7, 8, 9, 10]]

In [121]:
rdd.getNumPartitions()

4

In [124]:
rdd.coalesce(1).glom().collect()

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]

In [123]:
rdd1 = rdd.coalesce(1)
rdd1.getNumPartitions()

1


#### Spark Repartition() vs Coalesce()


Spark repartition() vs coalesce() – repartition() is used to increase or decrease the RDD, DataFrame, Dataset partitions whereas the coalesce() is used to only decrease the number of partitions in an efficient way.

In this article, you will learn what is Spark repartition() and coalesce() methods? and the difference between repartition vs coalesce with Scala examples.

NOTE: One important point to note is, Spark repartition() and coalesce() are very expensive operations as they shuffle the data across many partitions hence try to minimize repartition as much as possible.
#### Spark RDD repartition() vs coalesce()

In RDD, you can create parallelism at the time of the creation of an RDD using parallelize(), textFile() and wholeTextFiles(). You can download the test.txt file used in this example from GitHub.

In [126]:
rdd10 = sc.parallelize([i for i in range(0,100)], 4)
rdd10.getNumPartitions()

4

In [129]:
print(rdd10.glom().collect())

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74], [75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]]


#### RDD repartition()

Spark RDD repartition() method is used to increase or decrease the partitions. The below example decreases the partitions from 10 to 4 by moving data from all partitions.


This yields output Repartition size : 4 and the repartition re-distributes the data(as shown below) from all partitions which is full shuffle leading to very expensive operation when dealing with billions and trillions of data. 

In [131]:
rdd2 = rdd10.repartition(3)
rdd2.getNumPartitions()

3

#### RDD coalesce()

Spark RDD coalesce() is used only to reduce the number of partitions. This is optimized or improved version of repartition() where the movement of the data across the partitions is lower using coalesce.

In [132]:
print(rdd2.glom().collect())

[[20, 21, 22, 23, 24, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 70, 71, 72, 73, 74, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 95, 96, 97, 98, 99], [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84]]


In [135]:
rdd11 = rdd10.coalesce(2)
rdd11.getNumPartitions()

2

If you compared the below output with section 1, you will notice partition 3 has been moved to 2 and Partition 6 has moved to 5, resulting data movement from just 2 partitions.

In [151]:
##Cogroup
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2)])
[(x, tuple(map(list, y))) for x, y in sorted(list(x.cogroup(y).collect()))]

[('a', ([1], [2])), ('b', ([4], []))]

In [152]:
for x, y in sorted(list(x.cogroup(y).collect())):
    print((tuple(map(list,y))))

([1], [2])
([4], [])


#### pyspark.RDD.collectAsMap
This method should only be used if the resulting data is expected to be small, as all the data is loaded into the driver’s memory.

In [153]:
m = sc.parallelize([(1, 2), (3, 4)]).collectAsMap()

In [154]:
m

{1: 2, 3: 4}

In [155]:
m[1]

2

In case of duplicate keys

When we have duplicate keys, the latter key-value pair will overwrite the former ones:

In [163]:
m = sc.parallelize([(1, 2), (3, 4),(3, 19)]).collectAsMap()
m

{1: 2, 3: 19}

22/12/24 17:35:47 WARN HeartbeatReceiver: Removing executor 1 with no recent heartbeats: 131537 ms exceeds timeout 120000 ms
22/12/24 17:37:47 ERROR Utils: Uncaught exception in thread kill-executor-thread
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [120 seconds]. This timeout is controlled by spark.rpc.askTimeout
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.scheduler.cluster.CoarseGrainedSchedulerBackend.killExecutors(CoarseGrainedSchedulerBackend.scala:890)
	at org.apache.spark.SparkContext.killAndReplaceExecutor(SparkContext.s

22/12/24 17:45:06 WARN BlockManagerMasterEndpoint: Error trying to remove broadcast 140. The executor 1 may have been lost.
org.apache.spark.rpc.RpcTimeoutException: Cannot receive any reply from /192.168.87.202:44050 in 120 seconds. This timeout is controlled by spark.rpc.askTimeout
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at scala.util.Failure.recover(Try.scala:234)
	at scala.concurrent.Future.$anonfun$recover$1(Future.scala:395)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	a

22/12/24 17:49:25 ERROR YarnClientSchedulerBackend: YARN application has exited unexpectedly with state FAILED! Check the YARN application logs for more details.
22/12/24 17:49:25 ERROR YarnClientSchedulerBackend: Diagnostics message: Uncaught exception: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.deploy.yarn.ApplicationMaster.runExecutorLauncher(ApplicationMaster.scala:558)
	at org.apache.spark.deploy.yarn.ApplicationMaster.run(ApplicationMaster.scala:277)
	at org.apache.spark.deploy.yarn.ApplicationMaster$$anon$3.run(ApplicationMaster.scala:926)
	at org.apache.spark.deploy.yarn.ApplicationMaster$$anon$3.run(ApplicationMaster.scala:925)
	at java.security.AccessCont

#### pyspark.RDD.collect
This method should only be used if the resulting data is expected to be small, as all the data is loaded into the driver’s memory.

In [156]:
m = sc.parallelize([(1, 2), (3, 4)]).collect()

In [159]:
m

[(1, 2), (3, 4)]

In [162]:
m[0]

(1, 2)

https://www.skytowner.com/explore/pyspark_dataframe_alias_method

#### CombineByKey

Generic function to combine the elements for each key using a custom set of aggregation functions.

Turns an RDD[(K, V)] into a result of type RDD[(K, C)], for a “combined type” C.

Users provide three functions:

        createCombiner, which turns a V into a C (e.g., creates a one-element list)

        mergeValue, to merge a V into a C (e.g., adds it to the end of a list)

        mergeCombiners, to combine two C’s into a single one (e.g., merges the lists)

To avoid memory allocation, both mergeValue and mergeCombiners are allowed to modify and return their first argument instead of creating a new C.

In addition, users can control the partitioning of the output RDD.


#### Spark PairRDDFunctions: CombineByKey


Last time we covered one of the alternatives to the groupByKey function aggregateByKey. In this post we’ll cover another alternative PairRDDFunction – combineByKey. The combineByKey function is similar in functionality to the aggregateByKey function, but is more general. But before we go into details let’s review why we’d even want to avoid using groupByKey. When working in a map-reduce framework such Spark or Hadoop one of the steps we can take to ensure maximum performance is to limit the amount of data sent accross the network during the shuffle phase. The best option is when all operations can be performed on the map-side exclusively, meaning no data is sent at all to reducers. In most cases though, it’s not going to be realistic to do map-side operations only. If you need to do any sort of grouping, sorting or aggregation you’ll need to send data to reducers. But that doesn’t mean we still can’t attempt to make some optimizations.
GroupByKey Is Expensive

In the case of a groupByKey call, every single key-value pair will be shuffled accross the network with identical keys landing on the same reducer. To state the obvious, when grouping by key, the need for all matching keys to end up on the same reducer can’t be avoided. But one optimization we can attempt is to combine/merge values so we end up sending fewer key-value pairs in total. Addtionaly, less key-value pairs means reducers won’t have as much work to do, leading to additional performance gains. The groupByKey call makes no attempt at merging/combining values, so it’s an expensive operation.
CombineByKey

The combineByKey call is just such an optimization. When using combineByKey values are merged into one value at each partition then each partition value is merged into a single value. It’s worth noting that the type of the combined value does not have to match the type of the original value and often times it won’t be. The combineByKey function takes 3 functions as arguments:

A function that creates a combiner. In the aggregateByKey function the first argument was simply an initial zero value. In combineByKey we provide a function that will accept our current value as a parameter and return our new value that will be merged with addtional values.

The second function is a merging function that takes a value and merges/combines it into the previously collecte value(s).

The third function combines the merged values together. Basically this function takes the new values produced at the partition level and combines them until we end up with one singular value.


https://www.linkedin.com/pulse/spark-pyspark-combinebykey-simplified-swadeep-mishra-1/

In [10]:
x = sc.parallelize([("a", 1), ("b", 1), ("a", 2)])
x.collect()

[('a', 1), ('b', 1), ('a', 2)]

In [11]:

def to_list(a):
    return [a]


def append(a, b):
    a.append(b)
    return a


def extend(a, b):
    a.extend(b)
    return a


In [12]:
sorted(x.combineByKey(to_list, append, extend).collect())

[('a', [1, 2]), ('b', [1])]

In [15]:
print([1,2,3].extend([1]))

None


In [19]:
complexRDD.take(2)

[[('filamentA', '100W'), 605], [('filamentB', '100W'), 683]]

In [28]:
## Combinebykey sum
complexRDD.combineByKey(createCombiner=(lambda data: (data, 1)) \
                              , mergeValue=(lambda acc, data:(acc[0]+data, acc[1]+1)) \
                             ,mergeCombiners=(lambda data, data2:((data[0]+data2[0]), data[1]+data2[1]))).collect()

[(('filamentB', '100W'), (2756, 4)),
 (('filamentA', '200W'), (2170, 4)),
 (('filamentA', '100W'), (2514, 4)),
 (('filamentB', '200W'), (2285, 4))]

In [29]:
## Combinebykey sum
complexRDD.combineByKey(createCombiner=(lambda data: (data)) \
                       ,mergeValue=(lambda acc, data:(acc+data)) \
                       ,mergeCombiners=(lambda data, data2:(data+data2))).collect()

[(('filamentB', '100W'), 2756),
 (('filamentA', '200W'), 2170),
 (('filamentA', '100W'), 2514),
 (('filamentB', '200W'), 2285)]

In [31]:
## combinebykey avg
complexRDD.combineByKey(createCombiner=(lambda data:(data, 1)) \
                       ,mergeValue=(lambda acc, data:(acc[0]+data, acc[1]+1)) \
                       ,mergeCombiners=(lambda data, data2:((data[0]+data2[0])/(data[1]+data2[1])))).collect()

[(('filamentB', '100W'), 689.0),
 (('filamentA', '200W'), 542.5),
 (('filamentA', '100W'), 628.5),
 (('filamentB', '200W'), 571.25)]

In [32]:
## combinebykey min
complexRDD.combineByKey(createCombiner=(lambda data:data) \
                       ,mergeValue=(lambda acc, data:(min(acc, data))) \
                       ,mergeCombiners=(lambda data, data2: min(data, data2))).collect()

[(('filamentB', '100W'), 683),
 (('filamentA', '200W'), 520),
 (('filamentA', '100W'), 605),
 (('filamentB', '200W'), 555)]

In [33]:
##combinebykey max
complexRDD.combineByKey(createCombiner=(lambda data:data) \
                       ,mergeValue=(lambda acc, data:max(acc, data)) \
                       ,mergeCombiners=(lambda data, data2: max(data, data2))).collect()

[(('filamentB', '100W'), 696),
 (('filamentA', '200W'), 579),
 (('filamentA', '100W'), 668),
 (('filamentB', '200W'), 600)]

#### pyspark.RDD.context

The SparkContext that this RDD was created on.



In [36]:
complexRDD.context

<SparkContext master=yarn appName=pyspark-shell>

#### pyspark.RDD.count

Return the number of elements in this RDD.

In [37]:
complexRDD.count()

16

In [38]:
x = sc.parallelize([1,2,3])
x.count()

3

#### pyspark.RDD.countApprox

Approximate version of count() that returns a potentially incomplete result within a timeout, even if not all tasks have finished.

Now coming back to the question of count() vs countApprox(). Count is just like doing a select count(*) from Table. countApprox can have a timeout and confidence level which returns back a result which is approximately correct and a number you can live with.

We should use countApprox when we are more interested in knowing an approximate number and save time for example in a streaming application. Count() should be used when you need the exact count for example to log something or for auditing.

In [2]:
x = sc.parallelize(range(1000),10)
print(x.collect())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [3]:
x.countApprox(timeout=1)

1000

#### pyspark.RDD.countApproxDistinct¶

Return approximate number of distinct elements in the RDD.

In [6]:
x.countApproxDistinct()

1006

#### pyspark.RDD.countByKey
Count the number of elements for each key, and return the result to the master as a dictionary.

In [8]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
rdd.collect()

[('a', 1), ('b', 1), ('a', 1)]

In [11]:
rdd.countByKey()

defaultdict(int, {'a': 2, 'b': 1})

In [12]:
rdd.countByKey()['a']

2

In [14]:
sorted(rdd.countByKey().items())

[('a', 2), ('b', 1)]

#### pyspark.RDD.countByValue
Return the count of each unique value in this RDD as a dictionary of (value, count) pairs.

In [15]:
rdd.countByValue()

defaultdict(int, {('a', 1): 2, ('b', 1): 1})

In [16]:
rdd1 = sc.parallelize([1,2,3,1,3,1,3])
rdd1.collect()

[1, 2, 3, 1, 3, 1, 3]

In [18]:
rdd1.countByValue().items()

dict_items([(1, 3), (2, 1), (3, 3)])

In [20]:
sorted(rdd1.countByValue().items())

[(1, 3), (2, 1), (3, 3)]

In [21]:
sc.parallelize(sorted(rdd1.countByValue().items())).collect()

[(1, 3), (2, 1), (3, 3)]

#### pyspark.RDD.distinct
Return a new RDD containing the distinct elements in this RDD.

In [23]:
rdd.distinct().collect()

[('a', 1), ('b', 1)]

In [24]:
rdd.distinct().count()

2

In [25]:
rdd1.distinct().collect()

[2, 1, 3]

In [31]:
filDataSingleRDD.collect()

[['filamentA', '100W', 605],
 ['filamentB', '100W', 683],
 ['filamentB', '100W', 691],
 ['filamentB', '200W', 561],
 ['filamentA', '200W', 530],
 ['filamentA', '100W', 619],
 ['filamentB', '100W', 686],
 ['filamentB', '200W', 600],
 ['filamentB', '100W', 696],
 ['filamentA', '200W', 579],
 ['filamentA', '200W', 520],
 ['filamentA', '100W', 622],
 ['filamentA', '100W', 668],
 ['filamentB', '200W', 569],
 ['filamentB', '200W', 555],
 ['filamentA', '200W', 541]]

In [32]:
filDataSingleRDD.count()

16

In [33]:
pairedrdd = filDataSingleRDD.map(lambda data:(data[1],data[2]))
pairedrdd.collect()

[('100W', 605),
 ('100W', 683),
 ('100W', 691),
 ('200W', 561),
 ('200W', 530),
 ('100W', 619),
 ('100W', 686),
 ('200W', 600),
 ('100W', 696),
 ('200W', 579),
 ('200W', 520),
 ('100W', 622),
 ('100W', 668),
 ('200W', 569),
 ('200W', 555),
 ('200W', 541)]

In [34]:
pairedrdd.distinct().collect()

[('200W', 561),
 ('100W', 686),
 ('100W', 696),
 ('200W', 579),
 ('100W', 622),
 ('100W', 668),
 ('200W', 569),
 ('200W', 555),
 ('200W', 541),
 ('100W', 605),
 ('100W', 683),
 ('100W', 691),
 ('200W', 530),
 ('100W', 619),
 ('200W', 600),
 ('200W', 520)]

In [35]:
pairedrdd.distinct().count()

16

In [37]:
# distinct will not work on complex rdd and just created rdd it will worked on paired rdd
complexRDD.distinct().collect()

22/12/25 15:19:14 WARN TaskSetManager: Lost task 0.0 in stage 43.0 (TID 136) (audacious executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-audacious/nm-local-dir/usercache/audacious/appcache/application_1671801536838_0006/container_1671801536838_0006_01_000002/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/tmp/hadoop-audacious/nm-local-dir/usercache/audacious/appcache/application_1671801536838_0006/container_1671801536838_0006_01_000002/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/audacious/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/audacious/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/audacious/.local/lib/python3.10/site-packages/pys

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 43.0 failed 4 times, most recent failure: Lost task 0.3 in stage 43.0 (TID 142) (audacious executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-audacious/nm-local-dir/usercache/audacious/appcache/application_1671801536838_0006/container_1671801536838_0006_01_000002/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/tmp/hadoop-audacious/nm-local-dir/usercache/audacious/appcache/application_1671801536838_0006/container_1671801536838_0006_01_000002/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/audacious/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/audacious/.lo

#### pyspark.RDD.filter
Return a new RDD containing only the elements that satisfy a predicate.

In [38]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10], 2)
rdd.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [39]:
rdd.filter(lambda data: data > 5).collect()

[6, 7, 8, 9, 10]

In [40]:
rdd.filter(lambda data: data%2 == 0).collect()

[2, 4, 6, 8, 10]

In [41]:
rdd.filter(lambda data: data<0).collect()

[]

In [42]:
stringrdd = sc.parallelize(['hi', 'hellos','okay no problem', 'yes yes'])
stringrdd.collect()

['hi', 'hellos', 'okay no problem', 'yes yes']

In [43]:
stringrdd.filter(lambda data: len(data) > 5).collect()

['hellos', 'okay no problem', 'yes yes']

In [44]:
stringrdd.filter(lambda data: len(data) < 5).collect()

['hi']

In [45]:
stringrdd.filter(lambda data: 'h' in data).collect()

['hi', 'hellos']

In [47]:
stringrdd.filter(lambda data: 's' in data).collect()

['hellos', 'yes yes']

In [48]:
pairedrdd.collect()

[('100W', 605),
 ('100W', 683),
 ('100W', 691),
 ('200W', 561),
 ('200W', 530),
 ('100W', 619),
 ('100W', 686),
 ('200W', 600),
 ('100W', 696),
 ('200W', 579),
 ('200W', 520),
 ('100W', 622),
 ('100W', 668),
 ('200W', 569),
 ('200W', 555),
 ('200W', 541)]

In [49]:
pairedrdd2 = filDataSingleRDD.map(lambda data:(data[0],data[2]))
pairedrdd2.collect()

[('filamentA', 605),
 ('filamentB', 683),
 ('filamentB', 691),
 ('filamentB', 561),
 ('filamentA', 530),
 ('filamentA', 619),
 ('filamentB', 686),
 ('filamentB', 600),
 ('filamentB', 696),
 ('filamentA', 579),
 ('filamentA', 520),
 ('filamentA', 622),
 ('filamentA', 668),
 ('filamentB', 569),
 ('filamentB', 555),
 ('filamentA', 541)]

In [51]:
pairedrdd.filter(lambda data: data[0] == '100W').collect()

[('100W', 605),
 ('100W', 683),
 ('100W', 691),
 ('100W', 619),
 ('100W', 686),
 ('100W', 696),
 ('100W', 622),
 ('100W', 668)]

In [52]:
pairedrdd.filter(lambda data: (data[0] == '100W') and (data[1] > 650)).collect()

[('100W', 683), ('100W', 691), ('100W', 686), ('100W', 696), ('100W', 668)]

In [54]:
filDataSingleRDD.filter(lambda data:((data[0] =='filamentB') or (data[1] == '200W')) and (data[2] < 550)).collect()

[['filamentA', '200W', 530],
 ['filamentA', '200W', 520],
 ['filamentA', '200W', 541]]

In [60]:
complexRDD.filter(lambda data: (data[0][0]=='filamentA') and (data[1]<530)).collect()

[[('filamentA', '200W'), 520]]

#### pyspark.RDD.first
Return the first element in this RDD.

In [61]:
rdd.first()

1

In [62]:
complexRDD.first()

[('filamentA', '100W'), 605]

In [63]:
pairedrdd.first()

('100W', 605)

In [64]:
#map 
rdd.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [65]:
rdd.map(lambda data: data*2).collect()

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

In [66]:
rdd.map(lambda data: data/2).collect()

[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]

In [67]:
rdd.map(lambda data: data+10).collect()

[11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [68]:
rdd.map(lambda data: "Hi - {}".format(data)).collect()

['Hi - 1',
 'Hi - 2',
 'Hi - 3',
 'Hi - 4',
 'Hi - 5',
 'Hi - 6',
 'Hi - 7',
 'Hi - 8',
 'Hi - 9',
 'Hi - 10']

In [69]:
stringrdd.map(lambda data: (data, len(data))).collect()

[('hi', 2), ('hellos', 6), ('okay no problem', 15), ('yes yes', 7)]

In [71]:
stringrdd.map(lambda data: data[1:]).collect()

['i', 'ellos', 'kay no problem', 'es yes']

In [72]:
stringrdd.map(lambda data: data.split()).collect()

[['hi'], ['hellos'], ['okay', 'no', 'problem'], ['yes', 'yes']]

In [74]:
filDataSingleRDD.map(lambda data: [(data[0], data[1]),data[2]]).collect()

[[('filamentA', '100W'), 605],
 [('filamentB', '100W'), 683],
 [('filamentB', '100W'), 691],
 [('filamentB', '200W'), 561],
 [('filamentA', '200W'), 530],
 [('filamentA', '100W'), 619],
 [('filamentB', '100W'), 686],
 [('filamentB', '200W'), 600],
 [('filamentB', '100W'), 696],
 [('filamentA', '200W'), 579],
 [('filamentA', '200W'), 520],
 [('filamentA', '100W'), 622],
 [('filamentA', '100W'), 668],
 [('filamentB', '200W'), 569],
 [('filamentB', '200W'), 555],
 [('filamentA', '200W'), 541]]

#### PySpark flatMap() Transformation

PySpark flatMap() is a transformation operation that flattens the RDD/DataFrame (array/map DataFrame columns) after applying the function on every element and returns a new PySpark RDD/DataFrame. In this article, you will learn the syntax and usage of the PySpark flatMap() with an example.
Now, let’s see with an example of how to apply a flatMap() transformation on RDD. In the below example, first, it splits each record by space in an RDD and finally flattens it. Resulting RDD consists of a single word on each record.

In [75]:
stringrdd.collect()

['hi', 'hellos', 'okay no problem', 'yes yes']

In [76]:
stringrdd.map(lambda data: data.split()).collect()

[['hi'], ['hellos'], ['okay', 'no', 'problem'], ['yes', 'yes']]

In [77]:
stringrdd.flatMap(lambda data: data.split()).collect()

['hi', 'hellos', 'okay', 'no', 'problem', 'yes', 'yes']

In [79]:
filDataSingleRDD.take(2)

[['filamentA', '100W', 605], ['filamentB', '100W', 683]]

In [81]:
filDataSingleRDD.flatMap(lambda data: [data[0], data[1]]).collect()

['filamentA',
 '100W',
 'filamentB',
 '100W',
 'filamentB',
 '100W',
 'filamentB',
 '200W',
 'filamentA',
 '200W',
 'filamentA',
 '100W',
 'filamentB',
 '100W',
 'filamentB',
 '200W',
 'filamentB',
 '100W',
 'filamentA',
 '200W',
 'filamentA',
 '200W',
 'filamentA',
 '100W',
 'filamentA',
 '100W',
 'filamentB',
 '200W',
 'filamentB',
 '200W',
 'filamentA',
 '200W']

In [82]:
filDataSingleRDD.flatMap(lambda data: [data[0], data[2]]).collect()

['filamentA',
 605,
 'filamentB',
 683,
 'filamentB',
 691,
 'filamentB',
 561,
 'filamentA',
 530,
 'filamentA',
 619,
 'filamentB',
 686,
 'filamentB',
 600,
 'filamentB',
 696,
 'filamentA',
 579,
 'filamentA',
 520,
 'filamentA',
 622,
 'filamentA',
 668,
 'filamentB',
 569,
 'filamentB',
 555,
 'filamentA',
 541]

#### pyspark.RDD.flatMapValues
Pass each value in the key-value pair RDD through a flatMap function without changing the keys; this also retains the original RDD’s partitioning.

In [6]:
x = sc.parallelize([("a", ["x", "y", "z"]), ("b", ["p", "r"])])
x.collect()

[('a', ['x', 'y', 'z']), ('b', ['p', 'r'])]

In [7]:
x.flatMapValues(lambda data: data).collect()

[('a', 'x'), ('a', 'y'), ('a', 'z'), ('b', 'p'), ('b', 'r')]

In [8]:
x.flatMap(lambda data: data).collect()

['a', ['x', 'y', 'z'], 'b', ['p', 'r']]

#### fold
In this tutorial, you will learn fold syntax, usage and how to use Spark RDD fold() function in order to calculate min, max, and a total of the elements

Since RDD’s are partitioned, the fold() function takes full advantage of it by first aggregating elements in each partition and then aggregating results of all partitions to get the final result. The result of this function is the same as this RDD type.

This function takes the following arguments –

zeroValue – Initial value to be used for each partition in folding, this value would be used to initialize the accumulator for each partition. we mostly use 0 for integer and Nil for collections.

op – an operator used to both accumulate results within a partition and combine results from all partitions

#####Points to Note

fold() is similar to reduce() except it takes a ‘Zero value‘ as an initial value for each partition.

fold() is similar to aggregate() with a difference; fold return type should be the same as this RDD element type whereas aggregation can return any type.
    
fold() also same as foldByKey() except foldByKey() operates on Pair RDD


In [9]:
rdd.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [14]:
from operator import add

In [22]:
rdd.fold(0, op=(lambda acc,data: acc+data))

55

In [23]:
rdd.fold(0, op=(lambda acc, data: max(acc, data)))

10

In [29]:
rdd.fold(100, op=(lambda acc, data: min(acc, data)))

1

#### pyspark.RDD.foldByKey

In [30]:
filDataSingleRDD.take(2)

[['filamentA', '100W', 605], ['filamentB', '100W', 683]]

In [31]:
paired_rdd = filDataSingleRDD.map(lambda data: (data[0], data[2]))
paired_rdd.take(2)

[('filamentA', 605), ('filamentB', 683)]

In [32]:
paired_rdd.foldByKey(0, func=(lambda acc, data: acc+data)).collect()

[('filamentB', 5041), ('filamentA', 4684)]

In [33]:
paired_rdd.foldByKey(0, func=(lambda acc, data: max(acc, data))).collect()

[('filamentB', 696), ('filamentA', 668)]

In [35]:
paired_rdd.foldByKey(1000, func=(lambda acc, data: min(acc, data))).collect()

[('filamentB', 555), ('filamentA', 520)]

#### pyspark.RDD.foreach
A very simple example would be rdd.foreach(print) which would print the value of each row in the RDD but not modify the RDD in any way.
In conclusion, Spark foreach() is an action operation of RDD, DataFrame, Dataset which doesn’t have any return type and is used to manipulate the accumulator and write any external data sources.

In [49]:
rdd.foreach(f=ff)

In [57]:
rdd.foreach(lambda data: data*2)

In [51]:
rdd.foreach(print)

In [58]:
def f(iterator):
    for x in iterator:
        print(x)

sc.parallelize([1, 2, 3, 4, 5]).foreachPartition(f)


#### pyspark.RDD.getCheckpointFile
Gets the name of the file to which this RDD was checkpointed

Not defined if RDD is checkpointed locally.



In [74]:
rdd.getCheckpointFile()

#### pyspark.RDD.getResourceProfile
The user specified profile or None if none were specified



In [75]:
rdd.getResourceProfile()

#### pyspark.RDD.getStorageLevel
Get the RDD’s current storage level.

In [76]:
rdd.getStorageLevel()

StorageLevel(False, False, False, False, 1)

In [77]:
print(rdd.getStorageLevel())

Serialized 1x Replicated


#### pyspark.RDD.glom
Return an RDD created by coalescing all elements within each partition into a list.

In [78]:
rdd.glom().collect()

[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]

In [79]:
rdd.getNumPartitions()

2

#### groupby
Similar to SQL GROUP BY clause, PySpark groupBy() function is used to collect the identical data into groups on DataFrame and perform count, sum, avg, min, max functions on the grouped data.

In [106]:
rdd.groupBy(lambda data: data>5).mapValues(list).collect()

[(False, [1, 2, 3, 4, 5]), (True, [6, 7, 8, 9, 10])]

In [84]:
data = rdd.groupBy(lambda data: data>5).collect()
print([(x, sorted(y)) for x, y in data])

[(False, [1, 2, 3, 4, 5]), (True, [6, 7, 8, 9, 10])]


In [87]:
data = rdd.groupBy(lambda data: data%2).collect()
[(x, sorted(y)) for x,y in data]

[(0, [2, 4, 6, 8, 10]), (1, [1, 3, 5, 7, 9])]

In [89]:
rdd.groupBy(lambda data: data%2).count()

2

In [88]:
filDataSingleRDD.take(2)

[['filamentA', '100W', 605], ['filamentB', '100W', 683]]

In [91]:
filDataSingleRDD.groupBy(lambda data: data[0]).count()

2

In [108]:
filDataSingleRDD.groupBy(lambda data: data[0]).mapValues(list).collect()

[('filamentB',
  [['filamentB', '100W', 683],
   ['filamentB', '100W', 691],
   ['filamentB', '200W', 561],
   ['filamentB', '100W', 686],
   ['filamentB', '200W', 600],
   ['filamentB', '100W', 696],
   ['filamentB', '200W', 569],
   ['filamentB', '200W', 555]]),
 ('filamentA',
  [['filamentA', '100W', 605],
   ['filamentA', '200W', 530],
   ['filamentA', '100W', 619],
   ['filamentA', '200W', 579],
   ['filamentA', '200W', 520],
   ['filamentA', '100W', 622],
   ['filamentA', '100W', 668],
   ['filamentA', '200W', 541]])]

In [110]:
filDataSingleRDD.groupBy(lambda data: data[0]).mapValues(len).collect()

[('filamentB', 8), ('filamentA', 8)]

In [92]:
filDataSingleRDD.groupBy(lambda data: data[2]).count()

16

#### pyspark.RDD.groupByKey
Group the values for each key in the RDD into a single sequence. Hash-partitions the resulting RDD with numPartitions partitions.

Notes: If you are grouping in order to perform an aggregation (such as a sum or average) over each key, using reduceByKey or aggregateByKey will provide much better performance.

In [94]:
paired_rdd.groupByKey().count()

2

In [101]:
paired_rdd.groupByKey().mapValues(max).collect()

[('filamentB', 696), ('filamentA', 668)]

In [102]:
paired_rdd.groupByKey().mapValues(min).collect()

[('filamentB', 555), ('filamentA', 520)]

In [103]:
paired_rdd.groupByKey().mapValues(sum).collect()

[('filamentB', 5041), ('filamentA', 4684)]

In [104]:
paired_rdd.groupByKey().mapValues(len).collect()

[('filamentB', 8), ('filamentA', 8)]

In [105]:
paired_rdd.groupByKey().mapValues(list).collect()

[('filamentB', [683, 691, 561, 686, 600, 696, 569, 555]),
 ('filamentA', [605, 530, 619, 579, 520, 622, 668, 541])]

#### pyspark.RDD.groupWith

In [111]:
w = sc.parallelize([("a", 5), ("b", 6)])

x = sc.parallelize([("a", 1), ("b", 4)])

y = sc.parallelize([("a", 2)])

z = sc.parallelize([("b", 42)])

[(x, tuple(map(list, y))) for x, y in sorted(list(w.groupWith(x, y, z).collect()))]
[('a', ([5], [1], [2], [])), ('b', ([6], [4], [], [42]))]

[('a', ([5], [1], [2], [])), ('b', ([6], [4], [], [42]))]

#### pyspark.RDD.id
A unique ID for this RDD (within its SparkContext)

In [114]:
rdd.id()

0

In [115]:
filDataSingleRDD.id()

2

#### pyspark.RDD.intersection
 Return the intersection of this RDD and another one. The output will not contain any duplicate elements, even if the input RDDs did.
Notes:This method performs a shuffle internally.

In [119]:
rdd1 = sc.parallelize([1,2,3,11,15])
rdd1.collect()

[1, 2, 3, 11, 15]

In [120]:
rdd.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [121]:
rdd.intersection(rdd1).collect()

[1, 2, 3]

#### mpyspark.RDD.isCheckpointed
Return whether this RDD is checkpointed and materialized, either reliably or locally.



In [124]:
rdd.is_checkpointed

True

In [125]:
rdd1.is_checkpointed

False

#### pyspark.RDD.isEmpty¶
 Returns true if and only if the RDD contains no elements at all.

Notes: An RDD may be empty even when it has at least 1 partition.

In [126]:
s = sc.parallelize([])
s.collect()

[]

In [127]:
s.isEmpty()

True

In [128]:
y = sc.parallelize([1])
y.collect()

[1]

In [129]:
y.isEmpty()

False

#### pyspark.RDD.join

    Return an RDD containing all pairs of elements with matching keys in self and other.

    Each pair of elements will be returned as a (k, (v1, v2)) tuple, where (k, v1) is in self and (k, v2) is in other.

    Performs a hash join across the cluster

In [133]:
x = sc.parallelize([('a',1),('b',1)])
y = sc.parallelize([('a',10),('c',10)])

In [134]:
x.join(y).collect()

[('a', (1, 10))]

#### pyspark.RDD.keyBy
Creates tuples of the elements in this RDD by applying f.

In [135]:
x = sc.parallelize(range(0,3)).keyBy(lambda x: x*x)
x.collect()

[(0, 0), (1, 1), (4, 2)]

In [136]:
sc.parallelize(range(0,3)).collect()

[0, 1, 2]

In [137]:
rdd1.collect()

[1, 2, 3, 11, 15]

In [138]:
rdd1.keyBy(lambda data: data**2).collect()

[(1, 1), (4, 2), (9, 3), (121, 11), (225, 15)]

In [180]:
paired_rdd = filDataSingleRDD.map(lambda data: (data[0], data[2]))
paired_rdd.take(2)

[('filamentA', 605), ('filamentB', 683)]

In [145]:
filDataSingleRDD.keyBy(lambda data: data[0]).collect()

[('filamentA', ['filamentA', '100W', 605]),
 ('filamentB', ['filamentB', '100W', 683]),
 ('filamentB', ['filamentB', '100W', 691]),
 ('filamentB', ['filamentB', '200W', 561]),
 ('filamentA', ['filamentA', '200W', 530]),
 ('filamentA', ['filamentA', '100W', 619]),
 ('filamentB', ['filamentB', '100W', 686]),
 ('filamentB', ['filamentB', '200W', 600]),
 ('filamentB', ['filamentB', '100W', 696]),
 ('filamentA', ['filamentA', '200W', 579]),
 ('filamentA', ['filamentA', '200W', 520]),
 ('filamentA', ['filamentA', '100W', 622]),
 ('filamentA', ['filamentA', '100W', 668]),
 ('filamentB', ['filamentB', '200W', 569]),
 ('filamentB', ['filamentB', '200W', 555]),
 ('filamentA', ['filamentA', '200W', 541])]

In [146]:
filDataSingleRDD.keyBy(lambda data: (data[0], data[1])).collect()

[(('filamentA', '100W'), ['filamentA', '100W', 605]),
 (('filamentB', '100W'), ['filamentB', '100W', 683]),
 (('filamentB', '100W'), ['filamentB', '100W', 691]),
 (('filamentB', '200W'), ['filamentB', '200W', 561]),
 (('filamentA', '200W'), ['filamentA', '200W', 530]),
 (('filamentA', '100W'), ['filamentA', '100W', 619]),
 (('filamentB', '100W'), ['filamentB', '100W', 686]),
 (('filamentB', '200W'), ['filamentB', '200W', 600]),
 (('filamentB', '100W'), ['filamentB', '100W', 696]),
 (('filamentA', '200W'), ['filamentA', '200W', 579]),
 (('filamentA', '200W'), ['filamentA', '200W', 520]),
 (('filamentA', '100W'), ['filamentA', '100W', 622]),
 (('filamentA', '100W'), ['filamentA', '100W', 668]),
 (('filamentB', '200W'), ['filamentB', '200W', 569]),
 (('filamentB', '200W'), ['filamentB', '200W', 555]),
 (('filamentA', '200W'), ['filamentA', '200W', 541])]

#### pyspark.RDD.lookup
Return the list of values in the RDD for key key. This operation is done efficiently if the RDD has a known partitioner by only searching the partition that the key maps to.

In [152]:
l = range(10)

rdd = sc.parallelize(zip(l, l), 10)

rdd.lookup(4)  

[4]

In [153]:
rdd.collect()

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9)]

In [154]:
paired_rdd.lookup('filamentA')

['100W', '200W', '100W', '200W', '200W', '100W', '100W', '200W']

#### pyspark.RDD.map
Return a new RDD by applying a function to each element of this RDD.

In [156]:
rdd1.collect()

[1, 2, 3, 11, 15]

In [157]:
rdd1.map(lambda data: data **3).collect()

[1, 8, 27, 1331, 3375]

#### pyspark.RDD.mapPartitions
Return a new RDD by applying a function to each partition of this RDD.


Similar to map() PySpark mapPartitions() is a narrow transformation operation that applies a function to each partition of the RDD, if you have a DataFrame, you need to convert to RDD in order to use it. mapPartitions() is mainly used to initialize connections once for each partition instead of every row, this is the main difference between map() vs mapPartitions(). It is a narrow transformation as there will not be any data movement/shuffling between partitions to perform the function.

Key Points of PySpark MapPartitions():
    It is similar to map() operation where the output of mapPartitions() returns the same number of rows as in input RDD.
    It is used to improve the performance of the map() when there is a need to do heavy initializations like Database connection.
    mapPartitions() applies a heavy initialization to each partition of RDD instead of each element of RDD.
    It is a Narrow transformation operation
    PySpark DataFrame doesn’t have this operation hence you need to convert DataFrame to RDD to use mapPartitions()

In [163]:
rdd = sc.parallelize(range(10),2)
rdd.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [172]:
def add(v):
    yield sum(v)

In [173]:
rdd.mapPartitions(add).collect()

[10, 35]

#### pyspark.RDD.mapValues
Pass each value in the key-value pair RDD through a map function without changing the keys; this also retains the original RDD’s partitioning.

In [190]:
rdd = sc.parallelize([('a',[1]),('b',[10]),('a',[133]),('b',[30])])
rdd.collect()

[('a', [1]), ('b', [10]), ('a', [133]), ('b', [30])]

In [191]:
rdd.mapValues(len).collect()

[('a', 1), ('b', 1), ('a', 1), ('b', 1)]

In [192]:
rdd.mapValues(sum).collect()

[('a', 1), ('b', 10), ('a', 133), ('b', 30)]

In [194]:
rdd.aggregateByKey(0, seqFunc=(lambda acc,data: acc+ data[0]), combFunc=(lambda data, data2:data+data2)).collect()

[('b', 40), ('a', 134)]

In [225]:
rdd1.collect()

[1, 2, 3, 11, 15]

In [227]:
paired_rdd.mapValues(lambda data: (data,1)).collect()

[('filamentA', (605, 1)),
 ('filamentB', (683, 1)),
 ('filamentB', (691, 1)),
 ('filamentB', (561, 1)),
 ('filamentA', (530, 1)),
 ('filamentA', (619, 1)),
 ('filamentB', (686, 1)),
 ('filamentB', (600, 1)),
 ('filamentB', (696, 1)),
 ('filamentA', (579, 1)),
 ('filamentA', (520, 1)),
 ('filamentA', (622, 1)),
 ('filamentA', (668, 1)),
 ('filamentB', (569, 1)),
 ('filamentB', (555, 1)),
 ('filamentA', (541, 1))]

In [226]:
paired_rdd.collect()

[('filamentA', 605),
 ('filamentB', 683),
 ('filamentB', 691),
 ('filamentB', 561),
 ('filamentA', 530),
 ('filamentA', 619),
 ('filamentB', 686),
 ('filamentB', 600),
 ('filamentB', 696),
 ('filamentA', 579),
 ('filamentA', 520),
 ('filamentA', 622),
 ('filamentA', 668),
 ('filamentB', 569),
 ('filamentB', 555),
 ('filamentA', 541)]

#### pyspark.RDD.max
Find the maximum item in this RDD.

In [195]:
rdd1.collect()

[1, 2, 3, 11, 15]

In [196]:
rdd1.max()

15

In [197]:
rdd1.mean()

6.4

In [198]:
rdd1.min()

1

In [200]:
rdd1.name()

#### PySpark partitionBy() 
is a function of pyspark.sql.DataFrameWriter class that is used to partition based on one or multiple columns while writing DataFrame to Disk/File system. It creates a sub-directory for each unique value of the partition column.

Creating disk level partitioning, speeds up further data reading when you filter by partitioning column.

In [205]:
pairs = sc.parallelize([1, 2, 3, 4, 2, 4, 1]).map(lambda x: (x, x))
pairs.collect()

[(1, 1), (2, 2), (3, 3), (4, 4), (2, 2), (4, 4), (1, 1)]

In [214]:
sets = pairs.partitionBy(3).glom().collect()

In [215]:
sets[0]

[(3, 3)]

In [216]:
sets[1]

[(1, 1), (4, 4), (4, 4), (1, 1)]

In [217]:
sets

[[(3, 3)], [(1, 1), (4, 4), (4, 4), (1, 1)], [(2, 2), (2, 2)]]

#### persist

https://sparkbyexamples.com/spark/spark-persistence-storage-levels/

In [218]:
from pyspark import StorageLevel

In [219]:
rdd1.persist(storageLevel=StorageLevel.MEMORY_AND_DISK)

ParallelCollectionRDD[235] at readRDDFromFile at PythonRDD.scala:274

In [221]:
rdd1.is_cached

True

In [223]:
rdd.persist(storageLevel=StorageLevel.MEMORY_AND_DISK_2)

ParallelCollectionRDD[327] at readRDDFromFile at PythonRDD.scala:274

In [224]:
paired_rdd.persist(storageLevel=StorageLevel.MEMORY_ONLY_2)

PythonRDD[313] at collect at /tmp/ipykernel_6404/4007458860.py:1

#### pyspark.RDD.reduce
Reduces the elements of this RDD using the specified commutative and associative binary operator. Currently reduces partitions locally.

In [230]:
rdd1.reduce(max)

15

In [233]:
rdd1.reduce(min)

1

In [234]:
from operator import add

In [235]:
rdd1.reduce(add)

32

In [236]:
filDataSingleRDD.map(lambda data: data[2]).reduce(add)

9725

In [237]:
filDataSingleRDD.map(lambda data: data[2]).reduce(max)

696

In [238]:
complexRDD.map(lambda data: data[1]).reduce(max)

696

In [239]:
rdd1.reduce(lambda data, data2: data+data2)

32

#### PySpark reduceByKey() 

transformation is used to merge the values of each key using an associative reduce function on PySpark RDD. It is a wider transformation as it shuffles data across multiple partitions and It operates on pair RDD (key/value pair).

When reduceByKey() performs, the output will be partitioned by either numPartitions or the default parallelism level. The Default partitioner is hash-partition.

In [242]:
paired_rdd.take(1)

[('filamentA', 605)]

In [245]:
paired_rdd.reduceByKey(add).collect()

[('filamentB', 5041), ('filamentA', 4684)]

In [246]:
paired_rdd.reduceByKey(lambda data, data2: data+data2).collect()

[('filamentB', 5041), ('filamentA', 4684)]

In [248]:
paired_rdd.reduceByKey(lambda data, data2: max(data, data2)).collect()

[('filamentB', 696), ('filamentA', 668)]

In [249]:
paired_rdd.reduceByKey(max).collect()

[('filamentB', 696), ('filamentA', 668)]

#### pyspark.RDD.repartition
 Return a new RDD that has exactly numPartitions partitions.
Can increase or decrease the level of parallelism in this RDD. Internally, this uses a shuffle to redistribute data. If you are decreasing the number of partitions in this RDD, consider using coalesce, which can avoid performing a shuffle.


In [251]:
rdd = rdd.repartition(3)
rdd.getNumPartitions()

3

In [252]:
rdd = rdd.repartition(1)
rdd.getNumPartitions()

1

#### pyspark.RDD.sample¶

    Return a sampled subset of this RDD.

    Parameters

        withReplacementbool

            can elements be sampled multiple times (replaced when sampled out)
        fractionfloat

            expected size of the sample as a fraction of this RDD’s size without replacement: probability that each element is chosen; fraction must be [0, 1] with replacement: expected number of times each element is chosen; fraction must be >= 0
        seedint, optional

            seed for the random number generator



In [255]:
filDataSingleRDD.sample(True, 0.15,220).collect()

[['filamentA', '100W', 619], ['filamentB', '200W', 555]]

#### pyspark.RDD.sampleByKey¶

    Return a subset of this RDD sampled by key (via stratified sampling). Create a sample of this RDD using variable sampling rates for different keys as specified by fractions, a key to sampling rate map.

In [260]:
paired_rdd.sampleByKey(True,{"filamentA": 0.2, "filamentB": 0.1}).collect()

[('filamentB', 561),
 ('filamentA', 619),
 ('filamentA', 619),
 ('filamentB', 696),
 ('filamentA', 520),
 ('filamentA', 541)]

#### pyspark.RDD.setName

In [261]:
rdd.setName('rdd')

rdd MapPartitionsRDD[432] at coalesce at NativeMethodAccessorImpl.java:0

In [262]:
rdd.name()

'rdd'

#### pyspark.RDD.sortBy

In [263]:
paired_rdd.take(2)

[('filamentA', 605), ('filamentB', 683)]

In [264]:
paired_rdd.sortBy(lambda data: data[0]).collect()

[('filamentA', 605),
 ('filamentA', 530),
 ('filamentA', 619),
 ('filamentA', 579),
 ('filamentA', 520),
 ('filamentA', 622),
 ('filamentA', 668),
 ('filamentA', 541),
 ('filamentB', 683),
 ('filamentB', 691),
 ('filamentB', 561),
 ('filamentB', 686),
 ('filamentB', 600),
 ('filamentB', 696),
 ('filamentB', 569),
 ('filamentB', 555)]

In [265]:
paired_rdd.sortBy(lambda data: data[1]).collect()

[('filamentA', 520),
 ('filamentA', 530),
 ('filamentA', 541),
 ('filamentB', 555),
 ('filamentB', 561),
 ('filamentB', 569),
 ('filamentA', 579),
 ('filamentB', 600),
 ('filamentA', 605),
 ('filamentA', 619),
 ('filamentA', 622),
 ('filamentA', 668),
 ('filamentB', 683),
 ('filamentB', 686),
 ('filamentB', 691),
 ('filamentB', 696)]

In [266]:
rdd1.sortBy(lambda data: data).collect()

[1, 2, 3, 11, 15]

In [267]:
rdd1.sortBy(lambda data: -data).collect()

[15, 11, 3, 2, 1]

#### pyspark.RDD.sortByKey

In [268]:
rdd2 = sc.parallelize([('a',10),('b',7),('a',10),('b',7),('a',10),('a',7),('d',10),('c',7)])

In [269]:
rdd2.sortByKey().collect()

[('a', 10),
 ('a', 10),
 ('a', 10),
 ('a', 7),
 ('b', 7),
 ('b', 7),
 ('c', 7),
 ('d', 10)]

In [270]:
complexRDD.sortByKey().collect()

[(('filamentA', '100W'), 605),
 (('filamentA', '100W'), 619),
 (('filamentA', '100W'), 622),
 (('filamentA', '100W'), 668),
 (('filamentA', '200W'), 530),
 (('filamentA', '200W'), 579),
 (('filamentA', '200W'), 520),
 (('filamentA', '200W'), 541),
 (('filamentB', '100W'), 683),
 (('filamentB', '100W'), 691),
 (('filamentB', '100W'), 686),
 (('filamentB', '100W'), 696),
 (('filamentB', '200W'), 561),
 (('filamentB', '200W'), 600),
 (('filamentB', '200W'), 569),
 (('filamentB', '200W'), 555)]

#### pyspark.RDD.stats

In [273]:
rdd.collect()

[('a', [1]), ('b', [10]), ('a', [133]), ('b', [30])]

In [274]:
rdd1.collect()

[1, 2, 3, 11, 15]

In [275]:
rdd1.stats()

(count: 5, mean: 6.4, stdev: 5.571355310873647, max: 15.0, min: 1.0)

#### pyspark.RDD.subtract
Return each value in self that is not contained in other.

In [277]:
x = sc.parallelize([("a", 1), ("b", 4), ("b", 5), ("a", 3)])
y = sc.parallelize([("a", 3), ("c", None)])

sorted(x.subtract(y).collect())

[('a', 1), ('b', 4), ('b', 5)]

#### pyspark.RDD.subtractByKey

In [278]:
x.subtractByKey(y).collect()

[('b', 4), ('b', 5)]

pyspark.RDD.sum

In [279]:
rdd1.sum()

32

#### pyspark.RDD.take¶

Take the first num elements of the RDD.

It works by first scanning one partition, and use the results from that partition to estimate the number of additional partitions needed to satisfy the limit.

Translated from the Scala implementation in RDD#take().

Notes :This method should only be used if the resulting array is expected to be small, as all the data is loaded into the driver’s memory.

In [281]:
rdd.take(2)

[('a', [1]), ('b', [10])]

#### pyspark.RDD.takeOrdered

Get the N elements from an RDD ordered in ascending order or as specified by the optional key function.

Notes: This method should only be used if the resulting array is expected to be small, as all the data is loaded into the driver’s memory.

In [282]:
sc.parallelize([10, 1, 2, 9, 3, 4, 5, 6, 7]).takeOrdered(6)

[1, 2, 3, 4, 5, 6]

In [283]:
sc.parallelize([10, 1, 2, 9, 3, 4, 5, 6, 7], 2).takeOrdered(6, key=lambda x: -x)

[10, 9, 7, 6, 5, 4]

#### pyspark.RDD.top

Get the top N elements from an RDD.

Notes: This method should only be used if the resulting array is expected to be small, as all the data is loaded into the driver’s memory.

It returns the list sorted in descending order.

In [284]:
filDataSingleRDD.top(2)

[['filamentB', '200W', 600], ['filamentB', '200W', 569]]

In [286]:
rdd1.top(2,key=str)

[3, 2]

#### pyspark.RDD.unpersist¶
Mark the RDD as non-persistent, and remove all blocks for it from memory and disk.

In [287]:
rdd.unpersist()

rdd MapPartitionsRDD[432] at coalesce at NativeMethodAccessorImpl.java:0

In [288]:
paired_rdd.unpersist()

PythonRDD[313] at collect at /tmp/ipykernel_6404/4007458860.py:1

#### pyspark.RDD.values
Return an RDD with the values of each tuple.

In [290]:
paired_rdd.values().collect()

[605,
 683,
 691,
 561,
 530,
 619,
 686,
 600,
 696,
 579,
 520,
 622,
 668,
 569,
 555,
 541]

In [291]:
rdd.values().collect()

[[1], [10], [133], [30]]

#### pyspark.RDD.zip

    Zips this RDD with another one, returning key-value pairs with the first element in each RDD second element in each RDD, etc. Assumes that the two RDDs have the same number of partitions and the same number of elements in each partition (e.g. one was made through a map on the other).

In [292]:
x = sc.parallelize(range(0,5))

y = sc.parallelize(range(1000, 1005))

x.zip(y).collect()

[(0, 1000), (1, 1001), (2, 1002), (3, 1003), (4, 1004)]

#### pyspark.RDD.zipWithIndex

Zips this RDD with its element indices.

The ordering is first based on the partition index and then the ordering of items within each partition. So the first item in the first partition gets index 0, and the last item in the last partition receives the largest index.

This method needs to trigger a spark job when this RDD contains more than one partitions.

In [293]:
sc.parallelize(["a", "b", "c", "d"], 3).zipWithIndex().collect()

[('a', 0), ('b', 1), ('c', 2), ('d', 3)]

In [294]:
filDataSingleRDD.zipWithIndex().collect()

[(['filamentA', '100W', 605], 0),
 (['filamentB', '100W', 683], 1),
 (['filamentB', '100W', 691], 2),
 (['filamentB', '200W', 561], 3),
 (['filamentA', '200W', 530], 4),
 (['filamentA', '100W', 619], 5),
 (['filamentB', '100W', 686], 6),
 (['filamentB', '200W', 600], 7),
 (['filamentB', '100W', 696], 8),
 (['filamentA', '200W', 579], 9),
 (['filamentA', '200W', 520], 10),
 (['filamentA', '100W', 622], 11),
 (['filamentA', '100W', 668], 12),
 (['filamentB', '200W', 569], 13),
 (['filamentB', '200W', 555], 14),
 (['filamentA', '200W', 541], 15)]

#### pyspark.RDD.zipWithUniqueId¶

Zips this RDD with generated unique Long ids.

Items in the kth partition will get ids k, n+k, 2*n+k, …, where n is the number of partitions. So there may exist gaps, but this method won’t trigger a spark job, which is different from zipWithIndex().

In [295]:
sc.parallelize(["a", "b", "c", "d", "e"], 3).zipWithUniqueId().collect()

[('a', 0), ('b', 1), ('c', 4), ('d', 2), ('e', 5)]

In [296]:
filDataSingleRDD.zipWithUniqueId().collect()

[(['filamentA', '100W', 605], 0),
 (['filamentB', '100W', 683], 2),
 (['filamentB', '100W', 691], 4),
 (['filamentB', '200W', 561], 6),
 (['filamentA', '200W', 530], 8),
 (['filamentA', '100W', 619], 10),
 (['filamentB', '100W', 686], 12),
 (['filamentB', '200W', 600], 14),
 (['filamentB', '100W', 696], 1),
 (['filamentA', '200W', 579], 3),
 (['filamentA', '200W', 520], 5),
 (['filamentA', '100W', 622], 7),
 (['filamentA', '100W', 668], 9),
 (['filamentB', '200W', 569], 11),
 (['filamentB', '200W', 555], 13),
 (['filamentA', '200W', 541], 15)]

22/12/27 11:48:30 WARN DataStreamer: Exception for BP-241050457-127.0.0.1-1670841776195:blk_1073744969_4151
java.net.SocketTimeoutException: 65000 millis timeout while waiting for channel to be ready for read. ch : java.nio.channels.SocketChannel[connected local=/127.0.0.1:56260 remote=/127.0.0.1:9866]
	at org.apache.hadoop.net.SocketIOWithTimeout.doIO(SocketIOWithTimeout.java:163)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:161)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:131)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:118)
	at java.io.FilterInputStream.read(FilterInputStream.java:83)
	at java.io.FilterInputStream.read(FilterInputStream.java:83)
	at org.apache.hadoop.hdfs.protocolPB.PBHelperClient.vintPrefixed(PBHelperClient.java:519)
	at org.apache.hadoop.hdfs.protocol.datatransfer.PipelineAck.readFields(PipelineAck.java:213)
	at org.apache.hadoop.hdfs.DataStreamer$ResponseProcessor.run(DataStreamer.

22/12/27 15:17:36 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.io.IOException: All datanodes [DatanodeInfoWithStorage[127.0.0.1:9866,DS-e7dcc0b2-0f10-46a2-9872-6f61756e1ab0,DISK]] are bad. Aborting...
	at org.apache.hadoop.hdfs.DataStreamer.handleBadDatanode(DataStreamer.java:1609)
	at org.apache.hadoop.hdfs.DataStreamer.setupPipelineInternal(DataStreamer.java:1543)
	at org.apache.hadoop.hdfs.DataStreamer.setupPipelineForAppendOrRecovery(DataStreamer.java:1529)
	at org.apache.hadoop.hdfs.DataStreamer.processDatanodeOrExternalError(DataStreamer.java:1305)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:668)
22/12/27 15:17:46 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.io.IOException: All datanodes [DatanodeInfoWithStorage[127.0.0.1:9866,DS-e7dcc0b2-0f10-46a2-9872-6f61756e1ab0,DISK]] are bad. Aborting...
	at org.apache.hadoop.hdfs.DataStreamer.handleBadDatanode(DataStreamer.java:1609)
	at org.apache.hadoop.hdf

22/12/27 15:19:16 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.io.IOException: All datanodes [DatanodeInfoWithStorage[127.0.0.1:9866,DS-e7dcc0b2-0f10-46a2-9872-6f61756e1ab0,DISK]] are bad. Aborting...
	at org.apache.hadoop.hdfs.DataStreamer.handleBadDatanode(DataStreamer.java:1609)
	at org.apache.hadoop.hdfs.DataStreamer.setupPipelineInternal(DataStreamer.java:1543)
	at org.apache.hadoop.hdfs.DataStreamer.setupPipelineForAppendOrRecovery(DataStreamer.java:1529)
	at org.apache.hadoop.hdfs.DataStreamer.processDatanodeOrExternalError(DataStreamer.java:1305)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:668)
22/12/27 15:19:16 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.io.IOException: All datanodes [DatanodeInfoWithStorage[127.0.0.1:9866,DS-e7dcc0b2-0f10-46a2-9872-6f61756e1ab0,DISK]] are bad. Aborting...
	at org.apache.hadoop.hdfs.DataStreamer.handleBadDatanode(DataStreamer.java:1609)
	at org.apache.hadoop.hdf

22/12/27 15:33:01 WARN LeaseRenewer: Failed to renew lease for [DFSClient_NONMAPREDUCE_-104603192_14] for 33 seconds.  Will retry shortly ...
java.net.ConnectException: Call From audacious/192.168.87.202 to localhost:9000 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:913)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:828)
	at org.apache.hadoop.ipc.Client.getRpcResponse(Client.java:1616)
	at org.apache.hadoop.ipc.Client.call(Client.java:1558)
	at org.apache.hadoop.ip

...

22/12/27 15:33:43 WARN LeaseRenewer: Failed to renew lease for [DFSClient_NONMAPREDUCE_-104603192_14] for 75 seconds.  Will retry shortly ...
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.SafeModeException): Cannot renew lease for DFSClient_NONMAPREDUCE_-104603192_14. Name node is in safe mode.
The reported blocks 0 needs additional 606 blocks to reach the threshold 0.9990 of total blocks 607.
The minimum number of live datanodes is not required. Safe mode will be turned off automatically once the thresholds have been reached. NamenodeHostName:localhost
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.newSafemodeException(FSNamesystem.java:1577)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkNameNodeSafeMode(FSNamesystem.java:1564)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.renewLease(FSNamesystem.java:4019)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.renewLease(NameNodeRpcServer.java:1167)
	at org.apache

22/12/27 15:33:46 WARN LeaseRenewer: Failed to renew lease for [DFSClient_NONMAPREDUCE_-104603192_14] for 78 seconds.  Will retry shortly ...
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.SafeModeException): Cannot renew lease for DFSClient_NONMAPREDUCE_-104603192_14. Name node is in safe mode.
The reported blocks 607 has reached the threshold 0.9990 of total blocks 607. The minimum number of live datanodes is not required. In safe mode extension. Safe mode will be turned off automatically in 27 seconds. NamenodeHostName:localhost
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.newSafemodeException(FSNamesystem.java:1577)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkNameNodeSafeMode(FSNamesystem.java:1564)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.renewLease(FSNamesystem.java:4019)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.renewLease(NameNodeRpcServer.java:1167)
	at org.apache.hadoop.hdfs.protocolPB

22/12/27 15:33:49 WARN LeaseRenewer: Failed to renew lease for [DFSClient_NONMAPREDUCE_-104603192_14] for 81 seconds.  Will retry shortly ...
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.SafeModeException): Cannot renew lease for DFSClient_NONMAPREDUCE_-104603192_14. Name node is in safe mode.
The reported blocks 607 has reached the threshold 0.9990 of total blocks 607. The minimum number of live datanodes is not required. In safe mode extension. Safe mode will be turned off automatically in 24 seconds. NamenodeHostName:localhost
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.newSafemodeException(FSNamesystem.java:1577)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkNameNodeSafeMode(FSNamesystem.java:1564)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.renewLease(FSNamesystem.java:4019)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.renewLease(NameNodeRpcServer.java:1167)
	at org.apache.hadoop.hdfs.protocolPB

22/12/27 15:33:52 WARN LeaseRenewer: Failed to renew lease for [DFSClient_NONMAPREDUCE_-104603192_14] for 84 seconds.  Will retry shortly ...
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.SafeModeException): Cannot renew lease for DFSClient_NONMAPREDUCE_-104603192_14. Name node is in safe mode.
The reported blocks 607 has reached the threshold 0.9990 of total blocks 607. The minimum number of live datanodes is not required. In safe mode extension. Safe mode will be turned off automatically in 21 seconds. NamenodeHostName:localhost
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.newSafemodeException(FSNamesystem.java:1577)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkNameNodeSafeMode(FSNamesystem.java:1564)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.renewLease(FSNamesystem.java:4019)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.renewLease(NameNodeRpcServer.java:1167)
	at org.apache.hadoop.hdfs.protocolPB

22/12/27 15:33:55 ERROR Client: Application application_1672038063177_0001 not found.
22/12/27 15:33:55 WARN Client: Failed to cleanup staging dir hdfs://localhost:9000/user/audacious/.sparkStaging/application_1672038063177_0001
org.apache.hadoop.hdfs.server.namenode.SafeModeException: Cannot delete /user/audacious/.sparkStaging/application_1672038063177_0001. Name node is in safe mode.
The reported blocks 607 has reached the threshold 0.9990 of total blocks 607. The minimum number of live datanodes is not required. In safe mode extension. Safe mode will be turned off automatically in 18 seconds. NamenodeHostName:localhost
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.newSafemodeException(FSNamesystem.java:1577)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkNameNodeSafeMode(FSNamesystem.java:1564)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.delete(FSNamesystem.java:3265)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.delete(NameNodeRpc